# Graph exploring

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
from pyld import jsonld
import json
import pandas as pd
import io
import tempfile
import os
import pyoxigraph
import requests
import advertools as adv
import logging 
from scrapy.utils.log import configure_logging

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256
from io import StringIO

import datasets
import pandas as pd
from datasets import load_dataset
import torch
# from relik import Relik
# from relik.inference.data.objects import RelikOutput
from tqdm import tqdm

import logging
logging.basicConfig(level=logging.INFO)
# logging.getLogger('relik').setLevel(logging.INFO)

In [2]:
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("requests").setLevel(logging.ERROR)
configure_logging(install_root_handler=False)
logging.getLogger("scrapy").setLevel(logging.ERROR)

In [3]:
store = pyoxigraph.Store()  #  memory store  
# store = pyoxigraph.Store(path="./store")  # disk based store
mime_type = "application/n-quads"   # application/n-triples or application/n-quads if you are loading those

In [4]:
store.clear()

In [5]:

sourcesOLD = [
    "../datastore/data/global_pm25_concentration_1998_2016/meta_json-ld_global_pm25_concentration_1998_2016.json",
    # "../datastore/data/ma_2018_svi_tract/meta_json-ld_ma_2018_svi_tract.json",
    # "../datastore/data/tz_1984_copernicus_avg_temp/meta_json-ld_tz_1984_copernicus_avg_temp.json",
    # "../datastore/data/us_2018_svi_county/meta_json-ld_us_2018_svi_county.json",
    # "../datastore/data/tz_2022_nbs/tz_2022_nbs_districts/meta_json-ld_tz_2022_nbs_districts.json",
    # "../datastore/data/tz_2022_nbs_magu_district/meta_json-ld_tz_2022_nbs_magu_district.json",
    # "../datastore/data/tz_magu_dem_srtm/meta_json-ld_tz_magu_dem_srtm.json",
    # "../datastore/data/tz_populated_places_osm/meta_json-ld_tz_populated_places_osm.json",
    # "../datastore/data/tz_regions_osm/meta_json-ld_tz_regions_osm.json"
]

In [6]:
# NOT USED
def custom_document_loader(url, options):
    """Custom document loader that returns empty context for invalid URLs"""
    if url == '@context':
        # Return an empty context document
        return {
            'contextUrl': None,
            'documentUrl': url,
            'document': {}
        }
    # For other URLs, you might want to handle them differently
    # or let the default loader handle them
    raise jsonld.JsonLdError(
        f'URL could not be loaded: {url}',
        'jsonld.LoadDocumentError',
        {'url': url}
    )

In [7]:
sources = [
    "./data/meta_json-ld_global_pm25_concentration_1998_2016.nq",
    "./data/meta_json-ld_ma_2018_svi_tract.nq",
    "./data/meta_json-ld_tz_1984_copernicus_avg_temp.nq",
    "./data/meta_json-ld_tz_2022_nbs_districts.nq",
    "./data/meta_json-ld_tz_2022_nbs_magu_district.nq",
    # "./data/meta_json-ld_tz_magu_dem_srtm.nq",
    "./data/meta_json-ld_tz_populated_places_osm.nq",
    "./data/meta_json-ld_tz_regions_osm.nq",
    "./data/meta_json-ld_us_2018_svi_county.nq"
]



In [8]:
# Read a JSON file and parse it
for s in sources:
    with open(s) as nq_file:
      store.load(nq_file, pyoxigraph.RdfFormat.N_QUADS, base_iri=None, to_graph=None)


# #Read a JSON file and parse it
# for s in sources:
#     doc_array = None
#     with open(s) as json_file:
#       doc_array = json.load(json_file)
#
#     for ind, x in enumerate(doc_array):
#         doc = json.loads((json.dumps(x)))  # x is a dict so we need to typecast
#         normalized = jsonld.normalize(doc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
#         ns = io.StringIO(normalized)
#         store.load(ns, mime_type, base_iri=None, to_graph=None)

In [9]:
## NOTE   schema:contentURL changed!!!

qtest = """	PREFIX schema: <https://schema.org/>
SELECT ?s ?o 
WHERE {
      ?s schema:description ?o .
 }
"""

q1 = list(store.query(qtest))

In [10]:
df1 = pd.DataFrame(q1, columns=["subject",  "object" ])

In [11]:
df1

,subject,object
0,_:ef1e28f3654c9c8a18360150fb5d3478,"""\""Persons in group quarters estimate MOE, 201..."
1,_:c7c767dfd48998f443e5cd4a389939df,"""\""Civilian non-institutionalized population w..."
2,_:daa787885aab2a3d7a23a787c47321eb,"""\""At household level (occupied housing units)..."
3,_:e79fc804e104514bd15824867847dfb0,"""\""Persons aged 65 and older estimate MOE, 201..."
4,_:c869048048c13277cc4ef682f4801db,"""\""Persons aged 17 and younger estimate MOE, 2..."
...,...,...
346,_:c9be4046c20752d80bbbafac781a2ae8,"""GIS Vector Geometery"""
347,_:c459fedb208ba044b9d00ff15d6dcc45,"""GIS Data Structure for Representation"""
348,_:e0dac577a012070fc3b8fc2f4e6ad12f,"""#########\n# GDSC ETL is performed in two ste..."
349,_:d46ada41cce52c10ece4e5b73adab05f,"""Direct from source"""


In [12]:
term_counts = df1['object'].value_counts().reset_index()

# Rename the columns for clarity
term_counts.columns = ['object', 'count']

In [20]:
term_counts.head(30)

,object,count
0,"""GIS Data Structure for Representation""",8
1,"""GIS Vector Geometery""",7
2,"""Direct from source""",6
3,"""\""Per capita income estimate, 2014-2018 ACS\""""",4
4,"""Social vulnerability refers to the potential ...",4
5,""" direct from source""",4
6,"""\""Per capita income estimate MOE, 2014-2018 A...",4
7,"""Direct from source.""",4
8,"""A raster dataset with yearly averages of the ...",3
9,"""\""Persons aged 17 and younger estimate MOE, 2...",2


## Visualization section

In [21]:
#  a == rdf:type  == <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>

qtypetype = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?connectedBy ?target ?sType ?tType
WHERE {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?connectedBy ?target .
}
"""

# qtypetype = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
# PREFIX schema: <https://schema.org/>
#
# SELECT DISTINCT ?source ?sType ?kw
# WHERE {
#     ?source a ?sType .
#     ?source schema:keywords ?kw .
# }
# """

q2 = list(store.query(qtypetype))

In [23]:
# df2 = pd.DataFrame(q2, columns=["source", "sType", "keyword"])
df2 = pd.DataFrame(q2, columns=["source", "connecteBy", "target", "sType", "tType"])


In [24]:
df2 = df2.astype(str) # convert oxigraph types to string

In [25]:
gml = GraphML()
gml.directed = False

for index, row in df2.iterrows():
    n1 = gml.add_node(row['source'])
    n1['type'] = row['sType']
    
    n2 = gml.add_node(row['target'])
    n2['type'] = "tType"
 
    e1 = gml.add_edge(n1, n2)

# Create a temporary file
with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
    temp_filename = temp_file.name
    # Write the GraphML content to the temporary file
    parser = GraphMLParser()
    parser.write(gml, temp_filename)
    
g = nx.read_graphml(temp_filename)

os.remove(temp_filename)


In [30]:
# optional write the GML out
# fname = "./rdfGraphs2.graphml"
# parser = GraphMLParser()
# parser.write(gml, fname)


In [28]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)


Sigma(nx.Graph with 469 nodes and 469 edges)

## test relik against the descriptions

In [18]:
def clean_memory():
    # clean up a bit
    import gc

    relik = None
    gc.collect()
    torch.cuda.empty_cache()

In [19]:
def my_function(row):
  # Perform calculations or transformations on the row
  relik_out: RelikOutput = relik(row['description'])
  return relik_out

In [20]:
tqdm.pandas()

In [21]:
clean_memory()

In [22]:
relik = Relik.from_pretrained(
    "sapienzanlp/relik-entity-linking-small",
    # device="cuda" if torch.cuda.is_available() else "cpu",
    # precision="fp16" if torch.cuda.is_available() else "fp32",
    device="cpu", 
    precision="fp32",  
    skip_metadata=True,  # don't load index metadata to keep low memory requirements
    verbose=False  # Add this line to suppress debug output
)

# 
# relik = Relik.from_pretrained(
#     "relik-ie/relik-relation-extraction-small",
#     device="cuda" if torch.cuda.is_available() else "cpu",
#     precision="fp16" if torch.cuda.is_available() else "fp32",
#     skip_metadata=True,  # don't load index metadata to keep low memory requirements
# )


                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            

[2024-09-09 16:00:26,059] [INFO] [relik.inference.annotator.from_pretrained:700] [PID:2164100] [RANK:0] Loading Relik from sapienzanlp/relik-entity-linking-small
[2024-09-09 16:00:26,061] [INFO] [relik.inference.annotator.from_pretrained:701] [PID:2164100] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'span': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'riccorl/retriever-relik-e5-small-entity-linking-aida-wikipedia-index',
        },
    },
    'metadata_fields': []

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


[2024-09-09 16:01:03,042] [INFO] [relik.inference.utils.load_reader:383] [PID:2164100] [RANK:0] Moving reader to `cpu`.
[2024-09-09 16:01:03,044] [INFO] [relik.inference.utils.load_reader:386] [PID:2164100] [RANK:0] Setting precision of reader to `torch.float32`.
[2024-09-09 16:01:03,053] [WARNING] [py.warnings._showwarnmsg:109] [PID:2164100] /home/fils/.conda/envs/dev/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



In [23]:

qdesc = """	PREFIX schema: <https://schema.org/>
SELECT ?s ?o 
WHERE {
      ?s schema:description ?o .
 }
"""

q3 = list(store.query(qdesc))

In [24]:
df3 = pd.DataFrame(q3, columns=["source", "description"])

In [25]:
df3 = df3.astype(str) # convert oxigraph types to string

In [26]:
df3

,source,description
0,_:ead5613fc726f598e268a3f3824c001,"""Abstract"""
1,_:d4ffff3334397bfb3e63138305cad674,"""HLA Genotyping"""
2,_:7c74ed2f1eb3aa0ee2778ba9ba1d8565,"""IEDB epitopes."""
3,_:558cb4f387e613f85154bf92710fe59,"""Not Provided"""
4,_:e3c230e71331a1c7ea1a5f29b9e485c4,"""Not Provided"""
...,...,...
4166,_:a08d9537ba7b3c74e98c81721338d963,"""Anopheles annularis s.l. and An. vagus adult ..."
4167,_:c17276c80c698d01e01bc51c213cf64f,"""Pooled male and female 30 days old, resistant..."
4168,_:491871b6073e53074615c1637216a5f5,"""Severe acute respiratory syndrome coronavirus..."
4169,_:26b7ed9039489ef390b2aef32fbbdc3c,"""Collection and species identification of vari..."


In [29]:
df_b = pd.DataFrame()

In [30]:
for index, row in tqdm(df3.iterrows(), total=df3.shape[0]):
  result_dict = my_function(row)
  df = pd.DataFrame(result_dict.spans)
  # df = pd.DataFrame(result_dict.triplets)
  df['iri'] = row['source']
  df_b = pd.concat([df_b, df], ignore_index=True)

100%|██████████| 4171/4171 [3:25:47<00:00,  2.96s/it]  


In [31]:
# takes 3.5 hrs to build
df_b.to_parquet('df_b.parquet')

In [33]:
count_series = df_b['label'].value_counts()



In [35]:
count_series.to_csv('value_counts.csv', header=True, index_label='Value')


In [34]:
print(count_series)


label
United States                142
Plasmodium falciparum        136
Anopheles gambiae            109
Toxoplasma gondii             94
World Health Organization     94
                            ... 
Parakou                        1
Malanville                     1
ABCB4                          1
Anopheles nili                 1
Meyerozyma guilliermondii      1
Name: count, Length: 1274, dtype: int64


In [32]:
df_b

,iri,start,end,label,text
0,_:32847b64efc38e7adece56bbdbf91c44,39.0,59.0,Allomyces macrogynus,Allomyces macrogynus
1,_:84ff7b5cda76a10b23ff74d47f7a8906,27.0,37.0,"Ada County, Idaho",Ada County
2,_:84ff7b5cda76a10b23ff74d47f7a8906,102.0,112.0,"Ada County, Idaho",Ada County
3,_:84ff7b5cda76a10b23ff74d47f7a8906,114.0,119.0,Idaho,Idaho
4,_:84ff7b5cda76a10b23ff74d47f7a8906,121.0,124.0,United States,USA
...,...,...,...,...,...
4608,_:a08d9537ba7b3c74e98c81721338d963,1.0,20.0,Anopheles annularis,Anopheles annularis
4609,_:a08d9537ba7b3c74e98c81721338d963,224.0,227.0,DDT,DDT
4610,_:a08d9537ba7b3c74e98c81721338d963,240.0,252.0,Deltamethrin,deltamethrin
4611,_:491871b6073e53074615c1637216a5f5,1598.0,1606.0,Severe acute respiratory syndrome,COVID-19
